In [6]:
import a_funciones as funciones  ###archivo de funciones propias
import pandas as pd ### para manejo de datos
import sqlite3 as sql
import joblib
import sys
import openpyxl ## para exportar a excel
import numpy as np

In [7]:
df= pd.read_csv("https://raw.githubusercontent.com/juantoro5/Analitica-para-RH-E7/main/2016.csv", sep=",")
df

,EmployeeID,Age,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,...,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
0,1,0.785714,0.178571,0.25,0.00,0.637546,0.111111,0.000000,0.000000,1.000000,...,1,0,1,0,0,0,0,0,1,0
1,2,0.309524,0.321429,0.00,0.00,0.167457,0.000000,0.857143,0.333333,0.500000,...,0,0,1,0,0,0,0,0,0,1
2,3,0.333333,0.571429,0.75,0.75,0.964666,0.111111,0.285714,1.000000,0.333333,...,0,0,0,0,0,1,0,0,1,0
3,4,0.476190,0.035714,1.00,0.50,0.385045,0.333333,0.000000,1.000000,0.833333,...,0,0,1,0,0,0,0,0,1,0
4,5,0.333333,0.321429,0.00,0.00,0.070195,0.444444,0.071429,0.666667,0.333333,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,0.571429,0.142857,0.75,0.00,0.264350,0.333333,0.428571,0.333333,0.833333,...,0,0,0,0,1,0,0,0,0,1
4406,4407,0.261905,0.035714,0.75,0.00,0.087941,0.222222,0.285714,0.000000,0.333333,...,0,0,0,0,1,0,0,1,0,0
4407,4408,0.166667,0.857143,0.25,0.25,0.141811,0.000000,0.642857,0.000000,0.666667,...,0,0,1,0,0,0,0,0,1,0
4408,4409,0.571429,0.607143,0.25,0.00,0.073144,0.000000,0.214286,0.333333,0.333333,...,1,0,0,0,1,0,0,1,0,0


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 31 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   EmployeeID                         4410 non-null   int64  
 1   Age                                4410 non-null   float64
 2   DistanceFromHome                   4410 non-null   float64
 3   Education                          4410 non-null   float64
 4   JobLevel                           4410 non-null   float64
 5   MonthlyIncome                      4410 non-null   float64
 6   NumCompaniesWorked                 4410 non-null   float64
 7   PercentSalaryHike                  4410 non-null   float64
 8   StockOptionLevel                   4410 non-null   float64
 9   TrainingTimesLastYear              4410 non-null   float64
 10  YearsSinceLastPromotion            4410 non-null   float64
 11  EnvironmentSatisfaction            4410 non-null   float

In [8]:
####Otras transformaciones en python (imputación, dummies y seleccion de variables)


if __name__=="__main__":
    
    df_t= funciones.preparar_datos(df)


##Cargar modelo y predecir
    xgb_model= joblib.load("modelo_ganador.pkl")
    predicciones=xgb_model.predict(df_t)
    pd_pred=pd.DataFrame(predicciones, columns=['df_2016_Attrition'])


###Crear base con predicciones ####

    perf_pred=pd.concat([df_t['EmployeeID'],df_t,pd_pred],axis=1)
####LLevar a BD para despliegue 
    #perf_pred.loc[:,['EmpID2', 'pred_perf_2016']].to_sql("perf_pred",conn,if_exists="replace") ## llevar predicciones a BD con ID Empleados


####ver_predicciones_bajas ###
    emp_pred_bajo=perf_pred["df_2016_Attrition"]
    
    emp_pred_bajo.set_index('EmployeeID', inplace=True) 
    #pred=emp_pred_bajo.T
    
    #coeficientes=pd.DataFrame( np.append(m_lreg.intercept_,m_lreg.coef_) , columns=['coeficientes'])  ### agregar coeficientes
    emp_pred_bajo.to_excel("prediccion.xlsx")   #### exportar predicciones mas bajas y variables explicativas
    #coeficientes.to_excel("coeficientes.xlsx") ### exportar coeficientes para analizar predicciones

KeyError: "None of [Index(['BusinessTravel', 'Department', 'EducationField', 'JobRole',\n       'MaritalStatus'],\n      dtype='object')] are in the [columns]"

In [28]:
if __name__=="__main__":


    ### conectarse a la base de datos ###
    conn=sql.connect("data\\db_empleados")
    cur=conn.cursor()

    ### Ejecutar sql de preprocesamiento inicial y juntarlo 
    #### con base de preprocesamiento con la que se entrenó para evitar perdida de variables por conversión a dummies

    funciones.ejecutar_sql('preprocesamientos2.sql',cur) ### con las fechas actualizadas explicativas 2023- predecir 2024
    df=pd.read_sql('''select  * from base_completa2''',conn)

DatabaseError: Execution failed on sql '
        SELECT * FROM df_2016
        ': file is not a database

In [ ]:
#Preparar los datos para poder realizar las predicciones.
df_t= funciones.preparar_datos(df_2016)
df_t

In [ ]:
    ##Cargar modelo y predecir
    m_rfc = joblib.load("salidas\\rf_final.pkl")
    predicciones=m_rfc.predict(df_t)
    pd_pred=pd.DataFrame(predicciones, columns=['pred_renuncia_2017'])

In [ ]:

    ###Crear base con predicciones ####

    perf_pred=pd.concat([df_2016['EmployeeID'],df_t,pd_pred],axis=1)

In [ ]:
    m_dtc = joblib.load("modelo_ganador.pkl")
    predicciones=m_dtc.predict(df2)
    pd_prediction=pd.DataFrame(predicciones, columns=['Attrition'])